In [ ]:
import numpy as np

In [ ]:
from numba import jit

In [ ]:
ls ../result/

In [ ]:
@jit
def add (a, b, max_b):
    for i in range (a.shape[0]):
        for j in range (a.shape[1]):
            a[i][j] += b[i][j] / max_b

In [ ]:
#initial = np.load ("../result/BIGRAMS_BODY/ranking.npy")
#first = np.losd ("../result/BIGRAMS_TITLE/ranking.npy")

In [ ]:
initial = np.load ("../result/BIGRAMS_TITLE/ranking.npy")

In [ ]:
initial = initial / initial.max ()

In [ ]:
first = np.load ("../result/TRIGRAMS_TITLE/ranking.npy")

In [ ]:
add (initial, first, first.max ())

In [ ]:
del first

In [ ]:
second = np.load ("../result/WORDS_TITLE/ranking.npy")

In [ ]:
add (initial, second, second.max ())

In [ ]:
del second

In [ ]:
third = np.load ("../result/WORDS_BODY/ranking.npy")

In [ ]:
add (initial, third, third.max () * 2)

In [ ]:
del third

In [ ]:
fourth = np.load ("../result/BIGRAMS_BODY/ranking.npy")

In [ ]:
add (initial, fourth, fourth.max () * 5)

In [ ]:
del fourth

In [ ]:
fivth = np.load ("../result/TRIGRAMS_BODY/ranking.npy")

In [ ]:
add (initial, fivth, fivth.max () * 5)

In [ ]:
del fivth

In [ ]:
TRY_NAME = "whole"

In [ ]:
q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").upper ().split ("\t")
        q_ids.append (int (q[0]))
        #queries.append (dict (dct_title.doc2bow ([u''.join (gram) for gram in ngrams(q[1], NGRAMS, pad_left=True, pad_right=True, right_pad_symbol="$", left_pad_symbol="$")])))


In [ ]:
import tqdm

In [ ]:
with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (initial[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

np.save (open ("../result/{}/ranking.npy".format (TRY_NAME), "wb"), result)

In [ ]:
del initial


In [ ]:
from pymystem3 import Mystem

In [ ]:
title = True

In [ ]:
stemmer = Mystem ()

In [ ]:
with open ("../data/docs.tsv") as fin:
    with open ("../data/titles.tsv", "w") as title_doc:
        doc_num = 1 if title else 2
        
        for doc in tqdm.tqdm (fin):
            doc = doc.strip ().split("\t")
            
            result = doc[0] + '\t'
            if len (doc) > doc_num:
                result += ''.join (stemmer.lemmatize (doc[doc_num]))
                
            title_doc.write (result)

In [ ]:
title = False

In [ ]:
with open ("../data/docs.tsv") as fin:
    with open ("../data/bodies.tsv", "w") as title_doc:
        doc_num = 1 if title else 2
        
        for doc in tqdm.tqdm (fin):
            doc = doc.strip ().split("\t")
            
            result = doc[0] + '\t'
            if len (doc) > doc_num:
                result += ''.join (stemmer.lemmatize (doc[doc_num]))
                
            title_doc.write (result)

In [ ]:
logging.disable (logging.INFO)

In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora
from gensim import corpora, models, similarities
import numpy as np
import logging

TRY_NAME = "WORDS_TITLE_STEMMED"
title = True

dct = Dictionary(prune_at=None)

with open ("../data/titles.tsv") as fin:
    doc_num = 1 if title else 2
    
    for doc in tqdm.tqdm (fin):
        doc = doc.decode ("utf-8").lower ().strip ().split("\t")
        if len (doc) > doc_num:
            dct.add_documents ([doc[doc_num].split ()], prune_at=None)

dct.save ("../result/{}/dict.dct".format (TRY_NAME))

In [ ]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora
from gensim import corpora, models, similarities
import numpy as np
import logging
import gensim as gs
import math
import numpy as np
import tqdm

In [ ]:
TRY_NAME = "WORDS_TITLE_STEMMED"


In [ ]:
dct_title = gs.corpora.Dictionary.load ("../result/{}/dict.dct".format (TRY_NAME))

In [ ]:
q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").lower ().split ("\t")
        q_ids.append (int (q[0]))
        print ''.join (stemmer.lemmatize (q[1]))
        queries.append (dict (dct_title.doc2bow (stemmer.lemmatize (q[1]))))

In [ ]:
dct_title.doc2bow (["являться","ли"])

In [1]:
from gensim.corpora import Dictionary
import tqdm
import sys
from gensim import corpora
from gensim import corpora, models, similarities
import numpy as np
import logging

In [2]:
TRY_NAME = "WORDS_TITLE_STEMMED"
title = True

dct = Dictionary(prune_at=None)

with open ("../data/titles.tsv") as fin:
    doc_num = 1 
    
    for doc in tqdm.tqdm (fin):
        doc = doc.decode ("utf-8").lower ().strip ().split("\t")
        if len (doc) > doc_num:
            dct.add_documents ([doc[doc_num].split ()], prune_at=None)
        else:
            print "len == 0"

dct.save ("../result/{}/dict.dct".format (TRY_NAME))

558583it [00:14, 37791.74it/s]


In [4]:
# logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

def doc_reader (title):
    doc_num = 0
    
    with open ("../data/titles.tsv") as fin:
        for i, doc in enumerate (fin):
            doc = doc.decode ("utf-8").lower ().strip ().split("\t")[1:]
            if len (doc) > doc_num:
                yield dct.doc2bow(doc[doc_num].split ())
            else:
                print "len == 0"
                yield []

In [5]:
corpora.MmCorpus.serialize('../result/{}/corpus.mm'.format (TRY_NAME), doc_reader (title))

In [7]:
import gensim as gs
import math
import numpy as np
import tqdm

In [8]:
dct_title = gs.corpora.Dictionary.load ("../result/{}/dict.dct".format (TRY_NAME))

In [9]:
corpus = gs.corpora.MmCorpus('../result/{}/corpus.mm'.format (TRY_NAME))

PARAM_K1 = 1.5
PARAM_B = 0.75
EPSILON = 0.25

corp_size = 0
corp_size_words = 0
for doc in tqdm.tqdm (corpus):
    corp_size_words += sum (dict (doc).itervalues ())
    corp_size += 1

100%|██████████| 558583/558583 [00:03<00:00, 153142.26it/s]


In [11]:
from pymystem3 import Mystem
stemmer = Mystem ()

In [12]:
avgdl = float (corp_size_words) / corp_size

idfs = {}

summ_idf = 0

for word_id, freq in dct_title.dfs.iteritems ():
    idfs[word_id] = math.log(corp_size - freq + 0.5) - math.log(freq + 0.5)
    summ_idf += idfs[word_id]

average_idf = float (summ_idf) / len (dct_title.dfs)

for id in idfs.iterkeys ():
    idfs[id] = idfs[id] if idfs[id] >= 0 else EPSILON * average_idf

q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").lower ().split ("\t")
        q_ids.append (int (q[0]))
        queries.append (dict (dct_title.doc2bow (stemmer.lemmatize (q[1]))))

q_size = len (queries)

In [13]:
queries[0]

{752: 1, 872: 1, 4200: 1}

In [3]:
result = np.zeros ((q_size, corp_size))

for doc_i, doc in tqdm.tqdm (enumerate (corpus)):
    doc = dict (doc)
    doc_keys = set (doc)
    doc_len = sum (doc.itervalues ())
    for q_i, q in enumerate (queries):
        score = 0
        for word in set (q.keys ()) & doc_keys:
            idf = idfs[word]
            score += (idf * doc[word] * (PARAM_K1 + 1)
                      / (doc[word] + PARAM_K1 * (1 - PARAM_B + PARAM_B * doc_len / avgdl)))
        result[q_i, doc_i] = score

with open ("../result/result_{}.csv".format (TRY_NAME), "w") as fout:
    fout.write ("QueryId,DocumentId\n")
    
    for q_num, qid in tqdm.tqdm (enumerate (q_ids)):
        for doc in np.argsort (result[q_num, :])[-5:][::-1]:
            fout.write ("{},{}\n".format (qid, doc))

np.save (open ("../result/{}/ranking.npy".format (TRY_NAME), "wb"), result)

KeyboardInterrupt: 

In [ ]:
for item in dct_title.items ():
    print item[1]

In [ ]:
q_ids = []
queries = []
with open ("../data/queries.tsv") as fin:
    for q in fin:
        q = q.strip ().decode ("utf-8").lower ().split ("\t")
        q_ids.append (int (q[0]))
        print ''.join (stemmer.lemmatize (q[1]))
        queries.append (dict (dct_title.doc2bow (stemmer.lemmatize (q[1]))))